In [1]:
%load_ext literary.module

# Import Hook

In order to be able to import on-disk notebooks as Python modules, we need a Python import hook. This hook is lazy, as importing `nbconvert` et al. significantly affects startup time.

In [1]:
import sys
import traceback

from .finder import extend_file_finder, install_ipykernel_restrictor

In [2]:
def noop_loader(fullname, path):
    return None

We want to lazily load the notebook loader factory so that the Python startup is not adversely affected by importing nbconvert et al. 

Now we'll define a lazy proxy that ensures the import hook is fully loaded by the time a package needs it:

In [3]:
def notebook_loader_factory(fullname, path):
    try:
        get_loader = notebook_loader_factory.get_loader
    except AttributeError:
        # As we're installing the notebook hook, we shouldn't be loading any notebooks yet
        notebook_loader_factory.get_loader = noop_loader

        # Realise import of loader
        from .importer import get_loader

        notebook_loader_factory.get_loader = get_loader

    return get_loader(fullname, path)

Now we can install the import hook with a simple file finder

In [4]:
def install_import_hook(set_except_hook=True, restrict_ipykernel_path=True):
    if restrict_ipykernel_path:
        install_ipykernel_restrictor()

    extend_file_finder(
        (notebook_loader_factory, [".ipynb"]),
    )

    if set_except_hook:
        sys.excepthook = traceback.print_exception